<a href="https://colab.research.google.com/github/jessica157/Capstone-Project/blob/main/Capstone_Project_IBM_Jessica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install langchain_community
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.9 MB/s eta 0:00:00


In [7]:
from langchain_community.llms import Replicate
import os
from google.colab import userdata
# Set the API token
api_token = userdata.get('REPLICATE_API_TOKEN')
os.environ["REPLICATE_API_TOKEN"] = api_token
# Model setup
model = "ibm-granite/granite-3.3-8b-instruct"
output = Replicate(
    model=model,
    replicate_api_token=api_token,
)

## Data Classification

In [2]:
import pandas as pd

In [42]:
df = pd.read_csv('https://raw.githubusercontent.com/jessica157/Capstone-Project/refs/heads/main/student-por.csv')
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [44]:
# Define the customer reviews
student_grade = df['G1'].values

# Create formatted review text
grade = "\n".join([f"Classification {i+1}: {grade}" for i, grade in enumerate(student_grade)])

# Define the prompt
prompt = f"""
Classify each student scores into three categories: Great, Average, and Low.:\n
{grade}"""

# Invoke the model
response = output.invoke(prompt)

# Print the response
print("Granite Model Response:\n")
print(response)

Granite Model Response:

To classify the students' scores into Great, Average, and Low categories, we need to establish cutoff points. A common method is to use percentiles or quartiles. Here, I'll use a simple approach by setting the cutoffs at the 85th and 25th percentiles of the data. 

First, let's determine these percentiles using the given scores:

1. **Sort the scores**: The scores are already given in order.
2. **Find the 25th percentile (Q1)**: This is the score below which 25% of the data falls. 
   - \( Q1 = 9 \) (since 25 scores are less than or equal to 9)
3. **Find the 85th percentile (Q3)**: This is the score below which 85% of the data falls.
   - To find Q3, calculate the position: \( 0.85 \times (N + 1) = 0.85 \times 649 = 551.65 \). 
   - The 552nd score is just above 14.

Now, we can classify scores:

- **Great**: Scores above the 85th percentile (Q3), i.e., greater than 14.
- **Average**: Scores between the 25th and 85th percentiles (Q1 to Q3), i.e., between 10 and

In [80]:
# Define the customer reviews
parameters = {
"top_k": 0,
"top_p": 1.0,
"max_tokens": 256,
"min_tokens": 0,
"random_seed": None,
"repetition_penalty": 1.0,
"stopping_criteria": "length (256 tokens)",
"stopping_sequence": None
}

# Add initial prompt
refined_prompt = f""":
Classify these score as Great, Average, and Low.
Count the students for each classification.:\n
{grade}
"""
# Invoke the model
response = output.invoke(refined_prompt, parameters=parameters)
# Print the response
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

To classify the scores into Great, Average, and Low categories, we need to set thresholds for these classifications. A common approach is to divide the scores into three groups using percentiles or quartiles. Here, I'll use percentiles to classify the scores:

1. **Great**: Scores in the top 33% (the 8th percentile and above)
2. **Average**: Scores between the 33rd and 66th percentiles
3. **Low**: Scores below the 33rd percentile

Given the range of scores (4 to 18), let's determine the cutoffs:

- **Great**: 8th percentile ≈ 12
- **Average**: 33rd to 66th percentiles ≈ 11 to 13
- **Low**: Below the 33rd percentile ≈ 4 to 10

Now, let's classify and count the scores based on these cutoffs:

### Classification and Count:

1. **Great (12 and above):**
   - Count: 131 (Scores: 12, 13, 14, 15, 16, 17, 18, repeated several times)

2. **Average (11 and 10):**
   - Count: 140 (Scores: 10 appears 38 times, 11 appears 102 times)

3. **Low (4 to 9):**
   - Count:

In [87]:
student_grades = df[['G1','G2','G3']]

# Create formatted review text
grade = "\n".join([f"Classification {i+1}: {grade}" for i, grade in enumerate(student_grades)])

# Refine multiple Model Parameter values
parameters = {
"top_k": 1,
"top_p": 0.5,
"max_tokens": 10,
"min_tokens": 3,
"random_seed": None,
"repetition_penalty": 1.5,
"stopping_criteria": "length",
"stopping_sequence": None
}

# Add initial prompt
refined_prompt = f""":
Classify these as improved and downgrade.
Then count how many student for each classification.
The classification should be between G1 and G2, G2 and G3:\n
{grade}
"""
# Invoke the model
response = output.invoke(refined_prompt, parameters=parameters)
# Print the response
print("Granite Model Refined Response:\n")
print(response)

Granite Model Refined Response:

To classify the given classifications as improved (G1 to G2) or downgrade (G2 to G3), we need additional context or criteria that define what constitutes an improvement or a downgrade. Since no such criteria are provided, I'll illustrate how you might classify them based on a hypothetical improvement criterion, where moving from G1 to G2 is considered an improvement and moving from G2 to G3 is considered a downgrade.

### Hypothetical Classification Based on Improvement Criterion:

1. **Classification 1: G1**
   - **Hypothetical Classification**: Improved (G1 to G2)

2. **Classification 2: G2**
   - **Hypothetical Classification**: Not Improved/Neutral (remains G2)

3. **Classification 3: G3**
   - **Hypothetical Classification**: Downgrade (G2 to G3)

### Count of Classifications:

- **Improved (G1 to G2)**: 1 student
- **Downgrade (G2 to G3)**: 1 student
- **Neutral (G2 remains G2)**: 1 student

### Note:
Without specific criteria, these are assumptio

## Summarization

In [46]:
article = [
  """
  Autobiographical memory—our recollection of personal life events—is central to how we understand ourselves. “Memories are crucial for defining who we are,” says Hutmacher. “They are a sort of reservoir that we refer to whenever we think about our lives.”

Memory, however, isn’t like playing back a video. Neuroscientific research shows that this type of memory depends on interactions between the hippocampus, which helps consolidate new experiences, and the prefrontal cortex, which organizes them into coherent life narratives. These systems are especially sensitive to attention and emotional engagement—factors that may be disrupted when we’re more focused on photographing a moment than experiencing it.

“Our memory is not faithful,” says Julia Soares, assistant professor of psychology at Mississippi State University. “It’s tied up with who you are and your story making throughout your life. It’s your autobiography.”

Photos can support this reconstructive storytelling. Researchers agree that images often help jog memory by surfacing details or emotions we might otherwise forget. “I take them partly to document, but also as a way of holding onto moments I know I’ll want to revisit,” says 20-year-old Alina Nguyen. “It’s like a time capsule—sometimes I remember specific feelings or details I’d completely forgotten until I see an image…I think I’ve learned a lot about myself just by noticing patterns or changes in the photos over time.”

For Hutmacher, though, photos are more than just a cue for memories. He argues that in the digital age, photos are actually changing how we form memories in the first place. Remembering, he says, is no longer purely internal; it’s an interaction between our minds and resources like photos, meaning it’s built not only from what we store in our brains but also from what we offload onto devices like smartphones.

While offloading can reduce cognitive burden, studies suggest it may also weaken our ability to recall details unless we actively review the material later. As a result, when we turn to digital images to reconstruct an event, those files don’t just support our memory—they become part of it.

This shift raises new questions. If our memories are partly constructed through what we photograph—and what we choose to revisit—then our devices aren’t just reflecting our past. They’re shaping which moments we remember, how vividly, and supposedly how well we interpret our personal histories.

  """
]

In [47]:
# Refine the prompt to include reviews
summarize_text = "\n".join([f"Review {i+1}: {summarize}" for i, summarize in enumerate(article)])

In [51]:
# Apply default parameters
parameters = {
"top_k": 0,
"top_p": 1.0,
"max_tokens": 256,
"min_tokens": 0,
"random_seed": None,
"repetition_penalty": 1.0,
"stopping_criteria": "length (256 tokens)",
"stopping_sequence": None
}

# Refined prompt with example included
refined_focusarea_prompt = f"""
Summarize this meeting by focusing on key points, main idea, conclusion.
Make it into short and compact point.
{summarize_text}
"""

# Invoke the model with the refined focus prompt
response = output.invoke(refined_focusarea_prompt,parameters=parameters)
# Print the response
print("Granite Model Response for refined focus area response:\n")
print(response)

Granite Model Response for refined focus area response:

- **Key Points**:
  - Autobiographical memory, our recollection of personal experiences, is vital for self-understanding.
  - Memory formation involves the hippocampus (experience consolidation) and prefrontal cortex (organizing into life narratives), influenced by attention and emotional engagement.
  - Photos aid memory recall by triggering details and emotions, acting as a "time capsule" for personal reflection.
  - In the digital age, photos are not just memory aids but are reshaping how we form memories.
  - Offloading memories to digital devices reduces cognitive load but may weaken detail recall unless actively reviewed.
  - Digital images become integral to memory reconstructions, influencing which moments are remembered and how vividly.

- **Main Idea**: The interplay between autobiographical memory and photography in the digital era is transforming how we remember personal experiences, with photos not just assisting rec